# Santa

## import

In [141]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [142]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import random
import os
from scipy.spatial.distance import euclidean
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import seaborn as sns


# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [143]:
# Fixing the random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

# Set a fixed seed
fixed_seed = 42
set_seed(fixed_seed)

## 데이터 준비

In [150]:
# 데이터 준비
data_df = pd.read_csv('/content/drive/MyDrive/산타/data/data.csv')

# Load coordinates and demands from the data
towns = data_df[data_df['point_id'] != 'DEPOT']
depot = data_df[data_df['point_id'] == 'DEPOT']
depot_coords = (depot['x'].values[0], depot['y'].values[0])

In [151]:
# 최대 적재량
max_capacity = 25

# 도시 좌표 및 수요
town_coords = list(zip(towns['x'], towns['y']))
town_demands = towns['demand'].values
town_names = towns['point_id'].values

# 거리 행렬 생성 (depot 포함)
def calculate_distance_matrix(coords):
    """
    Calculate the distance matrix for all points, including the depot.
    """
    num_points = len(coords)
    distance_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance_matrix[i][j] = euclidean(coords[i], coords[j])
    return distance_matrix

# Distance callback function
def distance_callback(from_idx, to_idx, distance_matrix):
    return distance_matrix[from_idx][to_idx]

# 거리 행렬 생성 및 저장
town_coords.append(depot_coords)
distance_matrix = calculate_distance_matrix(town_coords)
depot_coords_idx = len(town_coords) - 1

## helper

In [152]:
# 경로를 수요 제약 조건을 만족하도록 나눕니다.
def split_route_by_capacity(route, demands, max_capacity):
    routes = []
    current_route = []
    current_capacity = max_capacity
    visited = set()

    for town_idx in route:
        if town_idx in visited or town_idx == depot_coords_idx:  # Depot 제외
            continue
        demand = demands[town_idx]
        if demand <= current_capacity:
            current_route.append(town_idx)
            current_capacity -= demand
            visited.add(town_idx)
        else:
            if current_route:
                routes.append(current_route)
            current_route = [town_idx]
            current_capacity = max_capacity - demand
            visited.add(town_idx)

    if current_route:
        routes.append(current_route)

    return routes


# 여러 sub-route의 총 DEPOT 왕복 거리를 계산.
def calculate_cvrp_cost(routes, distance_matrix, depot_idx, distance_callback):
    total_distance = 0
    for route in routes:
        if not route:
            continue
        # DEPOT -> first node
        total_distance += distance_callback(depot_idx, route[0], distance_matrix)
        # Between nodes
        for i in range(len(route) - 1):
            total_distance += distance_callback(route[i], route[i + 1], distance_matrix)
        # Last node -> DEPOT
        total_distance += distance_callback(route[-1], depot_idx, distance_matrix)
    return total_distance

# 전체 route의 총 거리를 계산. 여러 sub-route로 나누어 DEPOT 왕복 거리를 계산 후 합산.
def calculate_total_route_distance(route, distance_matrix, depot_idx):
    sub_routes = split_route_by_capacity(route, town_demands, max_capacity)
    total_distance = calculate_cvrp_cost(sub_routes, distance_matrix, depot_idx, distance_callback)
    return total_distance

# Nearest Neighbor Heuristic을 이용한 초기 경로 생성
def nearest_neighbor_initial_route(coords, depot_idx):
    num_towns = len(coords) - 1  # Exclude depot
    visited = [False] * len(coords)
    route = []

    current_idx = depot_idx
    visited[current_idx] = True

    while len(route) < num_towns:
        nearest_idx = -1
        nearest_distance = float('inf')

        for i in range(len(coords)):
            if not visited[i] and i != current_idx:
                distance = distance_matrix[current_idx][i]
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_idx = i

        if nearest_idx != -1:
            route.append(nearest_idx)
            visited[nearest_idx] = True
            current_idx = nearest_idx
        else:
            print(f"Failed to find a nearest unvisited town. Current route: {route}")
            break

    # Debug: Check for missing towns
    all_towns = set(range(len(coords) - 1))  # Exclude depot
    visited_towns = set(route)
    missing_towns = all_towns - visited_towns
    print(f"Generated route: {route}, Missing towns: {missing_towns}")

    return route


# 초기 population 생성. 수요 제약 조건을 만족하는 유효한 경로만 포함.
def initialize_population_with_nearest_neighbor(size, coords, demands, max_capacity, depot_idx):
    """
    Initialize the population using the Nearest Neighbor Heuristic.
    Ensures that all routes satisfy the capacity constraints.
    """
    population = []
    for _ in range(size):
        initial_route = nearest_neighbor_initial_route(coords, depot_idx)
        sub_routes = split_route_by_capacity(initial_route, demands, max_capacity)
        if all(sum(demands[idx] for idx in r) <= max_capacity for r in sub_routes):
            population.append(initial_route)

    if len(population) < size:
        raise ValueError("Unable to initialize population with given constraints using Nearest Neighbor.")

    return population



In [153]:
# 2-opt 알고리즘을 사용한 로컬 서치
def two_opt(route, distance_matrix, depot_coords_idx, max_iterations=10):
    best = route.copy()
    best_distance = calculate_total_route_distance(best, distance_matrix, depot_coords_idx)
    improved = True
    iteration = 0

    while improved and iteration < max_iterations:
        improved = False
        for i in range(1, len(best) - 2):
            for j in range(i + 1, len(best)):
                if j - i == 1:
                    continue
                new_route = best[:i] + best[i:j][::-1] + best[j:]
                new_distance = calculate_total_route_distance(new_route, distance_matrix, depot_coords_idx)
                if new_distance < best_distance:
                    best = new_route
                    best_distance = new_distance
                    improved = True
                    break  # Improvement found, return to the top
            if improved:
                break
        iteration += 1

    return best


# CVRP 경로 집합에 대해 로컬 서치.
def local_search_cvrp(routes, distance_matrix, depot_coords_idx):
    improved_routes = []
    for route in routes:
        improved_route = two_opt(route, distance_matrix, depot_coords_idx)
        improved_routes.append(improved_route)
    return improved_routes

# 일정 확률로 둘이 스왑
def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

# 토너먼트 선택
def tournament_selection(population, fitness, tournament_size=5):
    selected = random.sample(list(zip(population, fitness)), tournament_size)
    selected.sort(key=lambda x: x[1])  # Fitness 기준으로 정렬 (작을수록 좋음)
    return selected[0][0]  # 가장 좋은 해 반환

# Order Crossover (OX)
def ordered_crossover(parent1, parent2):
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))
    child_p1 = parent1[a:b]
    child_p2 = [item for item in parent2 if item not in child_p1]
    child = child_p1 + child_p2
    return child

# Fitness 계산 캐싱
fitness_cache = {}

def calculate_total_route_distance_with_cache(route, distance_matrix, depot_idx):
    """
    Calculate the total distance of a route with caching to avoid redundant calculations.
    """
    route_key = tuple(route)  # Use the route as a key for caching
    if route_key in fitness_cache:
        return fitness_cache[route_key]

    # If not in cache, calculate the distance
    distance = calculate_total_route_distance(route, distance_matrix, depot_idx)
    fitness_cache[route_key] = distance
    return distance

## GA

In [154]:
# Update genetic_algorithm_with_local_search to use the cached fitness calculation
def genetic_algorithm_with_local_search(coords, demands, depot_coords_idx, max_capacity,
                                        population_size, generations, mutation_rate,
                                        stop_threshold=100):
    num_towns = len(coords) - 1
    population = initialize_population_with_nearest_neighbor(
        size=population_size, coords=coords, demands=demands,
        max_capacity=max_capacity, depot_idx=depot_coords_idx
    )
    best_solution = None
    best_fitness = float('inf')

    no_improvement_count = 0

    for generation in range(generations):
        # Calculate fitness for each route in the population using the cache
        fitness = Parallel(n_jobs=-1)(
            delayed(calculate_total_route_distance_with_cache)(
                route, distance_matrix, depot_coords_idx
            ) for route in population
        )

        # Find the best solution in the current generation
        current_best_idx = np.argmin(fitness)
        current_best_fitness = fitness[current_best_idx]

        # Update the best solution
        if current_best_fitness < best_fitness:
            best_solution = population[current_best_idx]
            best_fitness = current_best_fitness
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Log progress
        if generation % 10 == 0 or generation == generations - 1:
            print(f"Generation {generation}: Best Distance = {best_fitness}")

        # Early stopping
        if no_improvement_count >= stop_threshold:
            break

        # Create new population using crossover and mutation
        new_population = []
        for _ in range(population_size):
            parent1 = tournament_selection(population, fitness)
            parent2 = tournament_selection(population, fitness)
            while parent2 == parent1:
                parent2 = tournament_selection(population, fitness)

            child = ordered_crossover(parent1, parent2)
            mutate(child, mutation_rate)
            new_population.append(child)

        population = new_population

    return best_solution, best_fitness


In [155]:
# 실행
population_size = 10
generations = 1000
mutation_rate = 0.1
stop_threshold = 100

best_route, best_distance = genetic_algorithm_with_local_search(
    coords=town_coords,
    demands=town_demands,
    depot_coords_idx=depot_coords_idx,
    max_capacity=max_capacity,
    population_size=population_size,
    generations=generations,
    mutation_rate=mutation_rate,
    stop_threshold=stop_threshold
)

final_routes = split_route_by_capacity(best_route, town_demands, max_capacity)
best_distance = calculate_cvrp_cost(final_routes, distance_matrix, depot_coords_idx, distance_callback)

print("Best Distance:", best_distance)
print("Routes:", final_routes)

Generated route: [9, 29, 10, 11, 30, 67, 36, 40, 31, 12, 28, 57, 44, 39, 49, 37, 38, 64, 35, 62, 25, 26, 27, 24, 74, 51, 58, 69, 50, 46, 6, 72, 5, 8, 41, 60, 18, 56, 68, 54, 52, 7, 71, 70, 65, 66, 59, 3, 2, 73, 0, 4, 43, 63, 42, 22, 13, 19, 20, 23, 15, 21, 14, 1, 17, 47, 33, 32, 34, 61, 53, 45, 16, 55, 48], Missing towns: set()
Generated route: [9, 29, 10, 11, 30, 67, 36, 40, 31, 12, 28, 57, 44, 39, 49, 37, 38, 64, 35, 62, 25, 26, 27, 24, 74, 51, 58, 69, 50, 46, 6, 72, 5, 8, 41, 60, 18, 56, 68, 54, 52, 7, 71, 70, 65, 66, 59, 3, 2, 73, 0, 4, 43, 63, 42, 22, 13, 19, 20, 23, 15, 21, 14, 1, 17, 47, 33, 32, 34, 61, 53, 45, 16, 55, 48], Missing towns: set()
Generated route: [9, 29, 10, 11, 30, 67, 36, 40, 31, 12, 28, 57, 44, 39, 49, 37, 38, 64, 35, 62, 25, 26, 27, 24, 74, 51, 58, 69, 50, 46, 6, 72, 5, 8, 41, 60, 18, 56, 68, 54, 52, 7, 71, 70, 65, 66, 59, 3, 2, 73, 0, 4, 43, 63, 42, 22, 13, 19, 20, 23, 15, 21, 14, 1, 17, 47, 33, 32, 34, 61, 53, 45, 16, 55, 48], Missing towns: set()
Generated 

KeyboardInterrupt: 

In [ ]:
# 경로 및 용량 확인
for route in final_routes:
    total_demand = sum(town_demands[idx] for idx in route)
    print(f"Route: {route}, Total Demand: {total_demand}, Capacity: {max_capacity}")
    assert total_demand <= max_capacity, "Route exceeds capacity!"

In [ ]:
# 모든 도시 리스트 (Depot 제외)
all_towns = set(range(len(town_coords) - 1))  # Depot 제외

# 초기 해 생성 후 확인
initial_routes = [nearest_neighbor_initial_route(town_coords, depot_coords_idx) for _ in range(10)]
for idx, route in enumerate(initial_routes):
    visited_towns = set(route)
    missing_towns = all_towns - visited_towns
    print(f"Initial Route {idx}: Missing towns: {missing_towns}")

# Capacity splitting 후 확인
final_routes = split_route_by_capacity(initial_routes[0], town_demands, max_capacity)
visited_towns = set()
for route in final_routes:
    visited_towns.update(route)
missing_towns = all_towns - visited_towns
print(f"After capacity splitting: Missing towns: {missing_towns}")


In [ ]:
# Visualization function
def visualize_routes(routes, town_coords, depot_coords, title="Optimal Routes"):
    plt.figure(figsize=(12, 12))

    # Extract town coordinates
    towns_x = [coord[0] for coord in town_coords]
    towns_y = [coord[1] for coord in town_coords]

    # Depot coordinates
    depot_x, depot_y = depot_coords

    # Plot towns and depot
    plt.scatter(towns_x, towns_y, c='blue', label='Towns', s=50)
    plt.scatter(depot_x, depot_y, c='red', label='Depot', s=100)

    # Filter valid routes
    valid_routes = [route for route in routes if route]
    colors = plt.cm.tab20(np.linspace(0, 1, len(valid_routes)))

    # Plot each route
    for idx, route in enumerate(valid_routes):
        # Route coordinates including depot as start and end
        route_coords = [depot_coords] + [town_coords[town_idx] for town_idx in route] + [depot_coords]
        xs, ys = zip(*route_coords)
        plt.plot(xs, ys, color=colors[idx], linestyle='-', linewidth=2, label=f'Route {idx + 1}')

    plt.title(title)
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# Visualize the provided routes
visualize_routes(final_routes, town_coords, depot_coords, title="Optimal Route Visualization")

## 파일 제출

In [136]:

# 제출 형식 맞추기
submission_route = ["DEPOT"]
current_capacity = max_capacity

for town_idx in best_route:
    if town_idx == depot_coords_idx:  # Depot은 건너뛴다
        continue

    demand = town_demands[town_idx]
    if current_capacity >= demand:
        submission_route.append(town_names[town_idx])
        current_capacity -= demand
    else:
        # 용량 초과 시 DEPOT으로 돌아감
        submission_route.append("DEPOT")
        submission_route.append(town_names[town_idx])
        current_capacity = max_capacity - demand

# 마지막에 DEPOT으로 복귀
submission_route.append("DEPOT")

# 제출 DataFrame 생성
submission_df = pd.DataFrame({"point_id": submission_route})

# 제출 파일 저장
submission_file_path = '/content/drive/MyDrive/santa_submission.csv'
submission_df.to_csv(submission_file_path, index=False)


In [137]:
# 파일 다운로드
from google.colab import files
files.download(submission_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>